# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [17]:
# Write your code below.
%load_ext dotenv
%dotenv C:\Users\Anna\Desktop\DSI\production\05_src\.env


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [18]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [42]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv('PRICE_DATA')
print(f'PRICE_DATA = {PRICE_DATA}')
parquet_files = glob(os.path.join(PRICE_DATA, "**/*parquet/part.0.parquet"))


PRICE_DATA = ../../05_src/data/prices/


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [43]:
# Read a parquet files into a Dask dataframe and sets the 'ticker'
dd_px = dd.read_parquet(parquet_files).set_index('ticker')

In [44]:
# Ensure the dataframe is sorted by ticker and date
dd_feat = dd.read_parquet(parquet_files).set_index('ticker')

In [45]:
# Add lags for variables Close and Adj_Close
dd_feat['Close_lag_1'] = dd_feat['Close'].shift(1)

In [47]:
# There isn't Adj_Close feature in this dataset
#dd_feat['Adj_Close_lag_1'] = dd_feat['Adj_Close'].shift(1)

In [48]:
# Add returns based on Adjusted Close
dd_feat['returns'] = dd_feat['Close'] / dd_feat['Close_lag_1'] - 1

In [49]:
# Add the hi_lo_range range
dd_feat['hi_lo_range'] = dd_feat['High'] - dd_feat['Low']

In [50]:
# Assign the results to dd_feat
dd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,returns,hi_lo_range
npartitions=11207,,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,int64,object,object,int32,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [51]:
# Save the dataframe as a Pandas dataframe
pd_feat = dd_feat.compute()


In [52]:
#Add a rolling average return calculation
pd_feat['rolling_avg_return_10d'] = pd_feat['returns'].rolling(window=10).mean()

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Answer: It wasn't necessary to convert the data to Pandas DataFrame to calculate the moving average return. Dask has .rolling() function similar to Pandas, so we could perform the operation directly in Dask. This would often be more efficient, especially with large datasets, because Dask uses parallel processing, which makes it faster than Pandas for such tasks. 

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.